<a href="https://colab.research.google.com/github/Shadrack25/myportfoloio/blob/master/stockanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import** **Libraries**

In [1]:
import json

#For DataFrame
import pandas as pd
import numpy as np

**Configuration**

In [2]:
#List of stock symbols we need to run fundamental analysis on- any symbol added here must have the json file containing stock info from yahoo finance
SYMBOLS = ['INTU','CDNS','WDAY','ROP','TEAM','ADSK','DDOG','ANSS','ZM','PTC','BSY','GRAB','SSNC','APP','AZPN','MANH','ZI','NICE' ]
#Path to read stock data from YF
DATA_PATH = 'path to access json files'
#Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY': [],
    'PB': [],
    'ROE': [],
    'P/S (trail)': [],
    'DPR': [],
    'DY': [],
    'CR': [],
    'Beta': [],
    'Price': [],
    '52W Low': [],
    '52W High': []
}

**Loads** **data**

In [3]:
#This is a utility method to extract indicators from the given json data and populate data dictionary
def load_data(json_data) :
  data['Symbol'].append(json_data['symbol'])
  data['Name'].append(json_data['longName'])
  data['Industry'].append(json_data['industry'])
  data['Price'].append(json_data['currentPrice'])

  #Could be that some indicators are not available; use Nan if this is the case

  if 'forwardEPS' in json_data:
    data['EPS (fwd)'].append(json_data['forwardEPS'])
  else:
    data['EPS (fwd)'].append(np.nan)

  if 'forwardPE' in json_data:
    data['P/E (fwd)'].append(json_data['forwardPE'])
  else:
    data['P/E (fwd)'].append(np.nan)

  if 'pegRatio' in json_data:
    data['PEG'].append(json_data['pegRatio'])
  else:
    data['PEG'].append(np.nan)

  if ('freeCashflow' in json_data) and ('marketCap' in json_data):
    fcfy = (json_data['freeCashflow']/json_data['marketCap']) * 100
    data['FCFY'].append(round(fcfy, 2))
  else:
    data['FCFY'].append(np.nan)

  if 'priceToBook' in json_data:
    data['PB'].append(json_data['priceToBook'])
  else:
    data['PB'].append(np.nan)

  if 'returnOnEquity' in json_data:
    data['ROE'].append(json_data['returnOnEquity'])
  else:
    data['ROE'].append(np.nan)

  if 'priceToSalesTrailing12Months' in json_data:
    data['P/S (trail)'].append(json_data['priceToSalesTrailing12Months'])
  else:
    data['P/S (trail)'].append(np.nan)

  if 'payoutRatio' in json_data:
    data['DPR'].append(json_data['payoutRatio'] * 100)
  else:
    data['DPR'].append(np.nan)

  if 'dividendYield' in json_data:
    data['DY'].append(json_data['dividendYield'])
  else:
    data['DY'].append(0.0)

  if 'beta' in json_data:
    data['Beta'].append(json_data['beta'])
  else:
    data['Beta'].append(np.nan)

  if 'currentRatio' in json_data:
    data['CR'].append(json_data['currentRatio'])
  else:
    data['CR'].append(np.nan)

  if 'fiftyTwoWeekLow' in json_data:
    data['52W Low'].append(json_data['fiftyTwoWeekLow'])
  else:
    data['52W Low'].append(np.nan)

  if 'fiftyTwoWeekHigh' in json_data:
    data['52W High'].append(json_data['fiftyTwoWeekHigh'])
  else:
    data['52W High'].append(np.nan)



**Load** **stock** **data** **from** **json** **files**

In [4]:
for symbol in SYMBOLS:
  #specify the full path to load JSON data
  file_name = f'{DATA_PATH}/{symbol}.json'
  try:
    #Open the fie in read mode
    with open(file_name, 'r') as file:
      #Use json.load() to parse the JSON DATA from the file
      load_data(json.load(file))
  except FileNotFoundError:
    print(f"File '{file_name}' not found.")
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON data: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")

#Loops throug the SYMBOLS declared in the configuration section and calls the load_dsata method to poplulate the data dictionary

File 'path to access json files/INTU.json' not found.
File 'path to access json files/CDNS.json' not found.
File 'path to access json files/WDAY.json' not found.
File 'path to access json files/ROP.json' not found.
File 'path to access json files/TEAM.json' not found.
File 'path to access json files/ADSK.json' not found.
File 'path to access json files/DDOG.json' not found.
File 'path to access json files/ANSS.json' not found.
File 'path to access json files/ZM.json' not found.
File 'path to access json files/PTC.json' not found.
File 'path to access json files/BSY.json' not found.
File 'path to access json files/GRAB.json' not found.
File 'path to access json files/SSNC.json' not found.
File 'path to access json files/APP.json' not found.
File 'path to access json files/AZPN.json' not found.
File 'path to access json files/MANH.json' not found.
File 'path to access json files/ZI.json' not found.
File 'path to access json files/NICE.json' not found.


**Create** **a** **Data** **Frame**
Any stock with NaN value for an indicator is moved to separate data frame.


In [11]:
#We also add a new column to allow for subsequent visualization via styles to show which stocks are near their 52week low and which are near 52week High
#A score of 90 indicates the present price is very near its 52 week high

#Create a DF using the dictionary
df = pd.DataFrame(data)
#Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]
#Remove any stocks with NaN values
df=df.dropna()
#Reset index after dropping rowswith Nan values
df.reset_index(drop=True, inplace=True)
#Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52W Low']) / (df['52W High'] - df['52W Low']))*100

df_exceptions

,symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52W Low,52W High


**Add** **styles**
A utility method to add styles to the data frame

In [12]:
def make_pretty(styler):
  #column formatting
  styler.format({'EPS (fwd)':'${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}', 'FCFY': '{:.2f}', 'PB': '{:.2f}', 'ROE': '{:.2f}', 'P/S (trail)': '{:.2f}',
                 'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52W Low': '${:.2f}', 'Price': '${:.2f}', '52W High': '${:.2f}',
                 '52w Range': '{:.2f}%'
  })
  #set the bar visualization
  styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

  #Grid
  styler.set_properties(** {'border': '0.1px solid black'})

  #Set background gradients
  styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
  styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
  styler.background_gradient(subset=['PEG'], cmap='Greens')
  styler.background_gradient(subset=['FCFY'],cmap='Greens')
  styler.background_gradient(subset=['PB'], cmap='Greens')
  styler.background_gradient(subset=['ROE'], cmap='Greens')
  styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
  styler.background_gradient(subset=['DPR'], cmap='Greens')
  styler.background_gradient(subset=['DY'], cmap='Greens')
  styler.background_gradient(subset=['CR'], cmap='Greens')

  #No index
  styler.hide(axis='index')

  #Tool tips
  styler.set_tooltips(
      ttips, css_class='tt-add',
      props=[
          ('visibility', 'hidden'),
          ('position', 'absolute'),
          ('background-color', 'salmon'),
          ('color', 'black'),
          ('z-index', 1),
          ('padding', '3px 3px'),
          ('margin', '2px')
      ]
  )

  #Left text alignment for some columns
  styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
  return styler

**Adds** **tool** **tips**
a utility method to add tool tips to the data frame created earlier

In [17]:
def populate_tt(df, tt_data, col_name):
  stats = df[col_name].describe()

  per25 = round(stats.loc['25%'], 2)
  per50 = round(stats.loc['50%'], 2)
  per75 = round(stats.loc['75%'], 2)

  #Get positions based on the column name
  pos = df.columns.to_list().index(col_name)

  for index, row in df.iterrows():
    pe = row[col_name]
    if pe == stats.loc['min']:
      tt_data[index][pos] = 'Lowest'
    elif pe == stats.loc['max']:
      tt_data[index][pos] = 'Highest'
    elif pe <= per25:
      tt_data[index][pos] = '25%of companies under {}'.format(per25)
    elif pe <=  per50:
      tt_data[index][pos] = '50% of companies under {}'.format(per50)
    elif pe <= per75:
      tt_data[index][pos] = '75% of companies under {}'.format(per75)
    else:
      tt_data[index][pos] = '25% of companies over {}'.format(per75)

    #Add stles and tool tips
    #Initialize tool tipdata - each column is set to '' for each row
    tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

    #Gather tool tip data for indicators
    populate_tt(df, tt_data, 'EPS (fwd)')
    populate_tt(df, tt_data, 'P/E (fwd)')
    populate_tt(df, tt_data, 'PEG')
    populate_tt(df, tt_data, 'FCFY')
    populate_tt(df, tt_data, 'PB')
    populate_tt(df, tt_data, 'ROE')
    populate_tt(df, tt_data, 'P/S (trail)')
    populate_tt(df, tt_data, 'DPR')
    populate_tt(df, tt_data, 'DY')
    populate_tt(df, tt_data, 'CR')

    #Create a tool tip DF
    ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

    #Add tablecaption and styles to DF
    df.style.pipe(make_pretty).set_Caption('Fundamental Indicators').set_table_styles(
        [{'selector': 'th.col_heading', 'props': ('text_align', 'center')}],
         {'selector':'caption', 'props': [('text-align', 'center'),
                                           ('font-size', '11pt'), ('font-weight', 'bold')]})


